In [1]:
import pandas as pd
import sqlite3
import warnings

warnings.filterwarnings('ignore')

In [2]:
#select directory that cointain db file
#%cd "C:\Users\Priscilla\Desktop\Gabriel"

connection = sqlite3.connect('desafio-tecnico.db')

cursor = connection.cursor()

#available
table_list = [a for a in cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

# here is you table list
print(table_list)

[('accounts',), ('address',), ('levels',), ('charges',), ('transactions',), ('transaction_type',)]


<h2>Querys

In [3]:
#Query for classification model

dfModel = pd.read_sql_query("""
    SELECT accounts.account_number,
    transactions.value as tansactionValue,
    description,
    accounts.occupation,
    date('now') - date(accounts.birth) as age,
    state,
    city,x
    
    FROM accounts
    
    left join transactions as transactions on transactions.account_number = accounts.account_number 
    left join transaction_type as transaction_type on transactions.transaction_type_id = transaction_type.id 
    left join levels as levels on accounts.account_number = levels.account_number 
    left join address as address on address.id = accounts.address_id
    
    WHERE tansactionValue <> '' and level IS NOT NULL"""
    ,connection)
dfModel.tail()

,account_number,tansactionValue,description,occupation,age,state,city,fraud
228919,700993,69457,pix_enviado,Corretor,36,ES,Sao Mateus,0
228920,947006,52144,pix_enviado,Investidor,56,RJ,Vassouras,0
228921,654870,18844,pix_enviado,Outros,22,MG,Extrema,0
228922,909353,22633,pix_enviado,Desenvolvedor,53,MG,Pirapora,0
228923,897231,81441,pix_enviado,Investidor,26,PR,Xambre,1


In [4]:
#Query of all accounts and transactions that do not have registered level
dfToClassify = pd.read_sql_query("""
    SELECT accounts.account_number,
    transactions.value as tansactionValue,
    description,
    accounts.occupation,
    date('now') - date(accounts.birth) as age,
    state,
    city,
    CASE level
        WHEN 'F' THEN '1'
        WHEN 'A' THEN '0'
        WHEN 'B' THEN '0'
        WHEN 'C' THEN '0'
        WHEN 'D' THEN '0'
    END fraud
    
    FROM accounts
    
    left join transactions as transactions on transactions.account_number = accounts.account_number 
    left join transaction_type as transaction_type on transactions.transaction_type_id = transaction_type.id 
    left join levels as levels on accounts.account_number = levels.account_number 
    left join address as address on address.id = accounts.address_id
    WHERE level IS NULL"""
    ,connection)

dfToClassify['tansactionValue'] = dfToClassify['tansactionValue'].fillna(0)
dfToClassify['description'] = dfToClassify['description'].fillna("0")

dfToClassify.tail()

,account_number,tansactionValue,description,occupation,age,state,city,fraud
27313,354382,3844.0,pix_recebido,Corretor,35,RJ,Sao Joao de Meriti,None
27314,354382,3893.0,pix_recebido,Corretor,35,RJ,Sao Joao de Meriti,None
27315,354382,4011.0,pix_recebido,Corretor,35,RJ,Sao Joao de Meriti,None
27316,354382,4188.0,pix_recebido,Corretor,35,RJ,Sao Joao de Meriti,None
27317,354382,4279.0,pix_recebido,Corretor,35,RJ,Sao Joao de Meriti,None


<h2> Pre-processing of Data

In [5]:
previsores = dfModel.iloc[:, 0:7].values
classe = dfModel.iloc[:, 7].values

from sklearn.preprocessing import LabelEncoder
label_encoder_acc_name = LabelEncoder()
label_encoder_level = LabelEncoder()
label_encoder_description = LabelEncoder()
label_encoder_occupation = LabelEncoder()
label_encoder_state = LabelEncoder()
label_encoder_city = LabelEncoder()

previsores[:,0] = label_encoder_acc_name.fit_transform(previsores[:,0])
previsores[:,2] = label_encoder_description.fit_transform(previsores[:,2])
previsores[:,3] = label_encoder_occupation.fit_transform(previsores[:,3])
previsores[:,5] = label_encoder_state.fit_transform(previsores[:,5])
previsores[:,6] = label_encoder_city.fit_transform(previsores[:,6])

from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.15, random_state=0)

previsores_treinamento.shape, classe_treinamento.shape
previsores_teste.shape, classe_teste.shape

((34339, 7), (34339,))

<h2> Application of classification models

<h3>Naive Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB
naive_risco_credito = GaussianNB()
naive_risco_credito.fit(previsores_treinamento, classe_treinamento)

previsoes = naive_risco_credito.predict(previsores_teste)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(classe_teste, previsoes)

0.9061417047671744

<h3> KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn_froud = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
knn_froud.fit(previsores_treinamento, classe_treinamento)
previsoes = knn_froud.predict(previsores_teste)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(classe_teste, previsoes)

0.9510469145869128

<h3> Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
logistic_froud = LogisticRegression(random_state=1)
logistic_froud.fit(previsores_treinamento, classe_treinamento)
previsoes = logistic_froud.predict(previsores_teste)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(classe_teste, previsoes)

0.8857858411718454

<h3> Redes Neurais Artificiais

In [9]:
from sklearn.neural_network import MLPClassifier
rede_neural_froud = MLPClassifier(max_iter=1500, verbose=True, tol=0.0000100,
                                   solver = 'adam', activation = 'relu',
                                   hidden_layer_sizes = (20,20))
rede_neural_froud.fit(previsores_treinamento, classe_treinamento)
previsoes = rede_neural_froud.predict(previsores_teste)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(classe_teste, previsoes)

Iteration 1, loss = 2.54478194
Iteration 2, loss = 1.56856876
Iteration 3, loss = 1.61789779
Iteration 4, loss = 1.31690771
Iteration 5, loss = 1.34778069
Iteration 6, loss = 1.17403439
Iteration 7, loss = 1.16193986
Iteration 8, loss = 1.18048960
Iteration 9, loss = 1.04648436
Iteration 10, loss = 1.17491603
Iteration 11, loss = 0.98646069
Iteration 12, loss = 1.07011753
Iteration 13, loss = 0.94008547
Iteration 14, loss = 0.96118172
Iteration 15, loss = 1.04026167
Iteration 16, loss = 1.02667966
Iteration 17, loss = 0.94987060
Iteration 18, loss = 0.85325025
Iteration 19, loss = 0.82435818
Iteration 20, loss = 0.81444757
Iteration 21, loss = 0.91789643
Iteration 22, loss = 0.88080833
Iteration 23, loss = 0.74499210
Iteration 24, loss = 0.87075198
Iteration 25, loss = 0.80953176
Iteration 26, loss = 0.66748531
Iteration 27, loss = 0.65113617
Iteration 28, loss = 0.72368400
Iteration 29, loss = 0.68953183
Iteration 30, loss = 0.64569182
Iteration 31, loss = 0.62141087
Iteration 32, los

0.9248958909694517

<h3> Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
random_forest_froud = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state = 0)
random_forest_froud.fit(previsores_treinamento, classe_treinamento)

previsoes = random_forest_froud.predict(previsores_teste)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(classe_teste, previsoes)

0.9999126357785608

<h3> Arvore de decisão

In [11]:
from sklearn.tree import DecisionTreeClassifier
arvore_risco_fraud = DecisionTreeClassifier(criterion='entropy')
arvore_risco_fraud.fit(previsores_treinamento, classe_treinamento)

previsoes = arvore_risco_fraud.predict(previsores_teste)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(classe_teste, previsoes)

0.9995923002999505

Arvore de Decisão e Random Forest se destacaram como modelos preditivos de idetificação.

A baixo o modelo sera aplicado aos usuários que ainda não possuem level classificado, de acordo com a query "dfToClassify".

In [12]:
print(dfToClassify['account_number'].count(),"Transações sem classificação de level")
print(dfToClassify['account_number'].nunique(),"Accounts com pendência de avaliação")

27318 Transações sem classificação de level
103 Accounts com pendência de avaliação


<h2> Pre-processing of data and application of classification model

In [13]:
atributos = dfToClassify.iloc[:, 0:7].values

from sklearn.preprocessing import LabelEncoder
label_encoder_acc_name = LabelEncoder()
label_encoder_level = LabelEncoder()
label_encoder_description = LabelEncoder()
label_encoder_occupation = LabelEncoder()
label_encoder_state = LabelEncoder()
label_encoder_city = LabelEncoder()

atributos[:,0] = label_encoder_acc_name.fit_transform(atributos[:,0])
atributos[:,2] = label_encoder_description.fit_transform(atributos[:,2])
atributos[:,3] = label_encoder_occupation.fit_transform(atributos[:,3])
atributos[:,5] = label_encoder_state.fit_transform(atributos[:,5])
atributos[:,6] = label_encoder_city.fit_transform(atributos[:,6])

#Classification of
previsoes = arvore_risco_fraud.predict(atributos)

<h3> Dataframe is created with all event classified

In [14]:
classification = pd.DataFrame()
classification['account_number'] = dfToClassify['account_number']
classification['fraud'] = previsoes

- After classifying the 27,318 missing records any user who has a transaction with fraud was classified as fraudulent.

In [31]:
classification['fraud'] = classification['fraud'].astype(int)

classificationAccount = pd.DataFrame((classification.groupby(classification['account_number'])['fraud'].max())).reset_index()
classificationAccount.to_csv("fraudClassification.csv",index = False, sep=';')